In [1]:
import networkx as nx
import pandas as pd
import logging
import pickle
import datetime

In [2]:
congresspeople = pd.read_csv('data/congresspeople/enriched_congresspeople.csv')
proposals = pd.read_csv('data/proposals/proposals.csv')
authors = pd.read_csv('data/authors/authors.csv')
authors = authors[authors['type'] == 'deputados']

In [3]:
def getUfRegion(uf):
    regions_dict = {
        "AM": "Norte",
        "RR": "Norte",
        "AP": "Norte",
        "PA": "Norte",
        "TO": "Norte",
        "RO": "Norte",
        "AC": "Norte",
        "MA": "Nordeste",
        "PI": "Nordeste",
        "CE": "Nordeste",
        "RN": "Nordeste",
        "PE": "Nordeste",
        "PB": "Nordeste",
        "SE": "Nordeste",
        "AL": "Nordeste",
        "BA": "Nordeste",
        "MT": "Centro-Oeste",
        "MS": "Centro-Oeste",
        "GO": "Centro-Oeste",
        "DF": "Distrito Federal",
        "SP": "Sudeste",
        "RJ": "Sudeste",
        "ES": "Sudeste",
        "MG": "Sudeste",
        "PR": "Sul",
        "RS": "Sul",
        "SC": "Sul"
    }
    return regions_dict[uf]

congresspeople['region'] = congresspeople['siglaUf'].apply(getUfRegion)

In [4]:
def getoccupation(occupation):
    occupation = occupation.upper()
    translation_table = {
       'DEPUTADO': 'politician',
       'SENADOR': 'politician',
       'VEREADOR': 'politician',
       'SENADOR, DEPUTADO E VEREADOR' : 'politician',
       'PRESIDENTE DA REPUBLICA, MINISTRO DE ESTADO, GOVERNADOR E PREFEITO' : 'politician',
       'OCUPANTE DE CARGO EM COMISSÃO' : 'politician',
       'ENGENHEIRO' : 'engineer/architect',
       'ARQUITETO' : 'engineer/architect',
       'SERVIDOR PUBLICO FEDERAL' : 'public servant',
       'SERVIDOR PÚBLICO FEDERAL' : 'public servant',
       'SERVIDOR PUBLICO ESTADUAL' : 'public servant',
       'SERVIDOR PÚBLICO ESTADUAL' : 'public servant',
       'SERVIDOR PÚBLICO ESTADUAL' : 'public servant',
       'SERVIDOR PUBLICO MUNICIPAL' : 'public servant',
       'SERVIDOR PÚBLICO MUNICIPAL' : 'public servant',
       'SERVIDOR PUBLICO CIVIL APOSENTADO' : 'public servant',
       'SERVIDOR PÚBLICO CIVIL APOSENTADO' : 'public servant',
       'TABELIÃO': 'public servant',
       'FISCAL': 'public servant',
       'MEMBRO DAS FORÇAS ARMADAS' : 'military',
       'MILITAR REFORMADO' : 'military',
       'BOMBEIRO MILITAR' : 'military',
       'POLICIAL MILITAR' : 'military',
       'POLICIAL CIVIL' : 'military',
       'PROFESSOR DE ENSINO SUPERIOR' : 'university professor',
       'PROFESSOR E INSTRUTOR DE FORMAÇÃO PROFISSIONAL' : 'teacher',
       'PROFESSOR E INSTRUTOR DE FORMACAO PROFISSIONAL' : 'teacher',
       'PROFESSOR DE ENSINO DE PRIMEIRO E SEGUNDO GRAUS' : 'teacher',
       'PROFESSOR DE ENSINO FUNDAMENTAL' : 'teacher',
       'PROFESSOR DE ENSINO MÉDIO' : 'teacher',
       'PEDAGOGO' : 'teacher',
       'ESTUDANTE, BOLSISTA, ESTAGIÁRIO E ASSEMELHADOS' : 'student',
       'ESTUDANTE, BOLSISTA, ESTAGIARIO E ASSEMELHADOS' : 'student',
       'ESTUDANTE,BOLSISTA, ESTAGIARIO E ASSEMELHADOS' : 'student',
       'MOTORISTA PARTICULAR' : 'driver',
       'MOTORISTA DE VEÍCULOS DE TRANSPORTE COLETIVO DE PASSAGEIROS' : 'driver',
       'OPERADOR DE EQUIPAMENTO DE RÁDIO, TELEVISÃO, SOM E CINEMA': 'media worker',
       'COMUNICÓLOGO': 'media worker',
       'COMUNICOLOGO': 'media worker',
       'LOCUTOR E COMENTARISTA DE RÁDIO E TELEVISÃO E RADIALISTA': 'media worker',
       'LOCUTOR E COMENTARISTA DE RADIO E TELEVISÃO E RADIALISTA': 'media worker',
       'LOCUTOR E COMENTARISTA DE RADIO E TELEVISAO E RADIALISTA': 'media worker',
       'FOTÓGRAFO E ASSEMELHADOS': 'media worker',
       'TRABALHADOR DE ARTES GRAFICAS': 'media worker',
       'TRABALHADOR DE ARTES GRÁFICAS': 'media worker',
       'JORNALISTA E REDATOR': 'journalist',
       'PUBLICITÁRIO': 'publicist',
       'ATOR E DIRETOR DE ESPETÁCULOS PÚBLICOS': 'actor/musician',
       'MÚSICO': 'actor/musician',
       'CANTOR E COMPOSITOR': 'actor/musician',
       'PECUARISTA': 'farmer', 
       'AGRICULTOR': 'farmer',
       'PRODUTOR AGROPECUARIO': 'farmer',
       'PRODUTOR AGROPECUÁRIO': 'farmer',
       'PESCADOR': 'farmer',
       'TECNICO CONTABILIDADE, ESTATISTICA, ECONOMIA DOMESTICA E ADMINISTRACAO': 'accountant',
       'TÉCNICO CONTABILIDADE, ESTATÍSTICA, ECONOMIA DOMÉSTICA E ADMINISTRAÇÃO': 'accountant',
       'CONTADOR': 'accountant',
       'VENDEDOR DE COMÉRCIO VAREJISTA E ATACADISTA': 'salesman',
       'COMERCIANTE': 'salesman',
       'REPRESENTANTE COMERCIAL': 'salesman',
       'COMERCIÁRIO': 'salesman',
       'INDUSTRIAL': 'blue collar',
       'OPERADOR DE APARELHOS DE PRODUÇÃO INDUSTRIAL': 'blue collar',
       'FERROVIÁRIO': 'blue collar',
       'FERROVIARIO': 'blue collar',
       'TRABALHADOR METALÚRGICO E SIDERÚRGICO': 'blue collar',
       'TRABALHADOR METALURGICO E SIDERURGICO': 'blue collar',
       'TECNICO DE ELETRICIDADE, ELETRONICA E TELECOMUNICACOES': 'blue collar',
       'ENCANADOR,SOLDADOR,CHAPEADOR,CALDEIREIRO,MONTADOR DE ESTRUTURA METALIC': 'blue collar',
       'TORNEIRO MECÂNICO': 'blue collar',
       'TÉCNICO EM EDIFICAÇÕES': 'blue collar',
       'VETERINÁRIO': 'veterinarian/agronomist',
       'VETERINARIO': 'veterinarian/agronomist',
       'ZOOTECNISTA': 'veterinarian/agronomist',
       'AGRONOMO': 'veterinarian/agronomist',
       'AGRÔNOMO': 'veterinarian/agronomist',
       'TÉCNICO EM AGRONOMIA E AGRIMENSURA': 'veterinarian/agronomist/',
       'OPERADOR DE IMPLEMENTO DE AGRICULTURA, PECUARIA E EXPLORACAO FLORESTAL': 'veterinarian/agronomist',
       'ODONTOLOGO': 'dentist/medic',
       'ODONTÓLOGO': 'dentist/medic',
       'MÉDICO': 'dentist/medic',
       'MEDICO': 'dentist/medic',
       'ADVOGADO': 'judge/lawyer/prosecutor',
       'BIOMÉDICO': 'biologist',
       'BIOLOGO E BIOMEDICO': 'biologist',
       'BIÓLOGO': 'biologist',
       'PSICOLOGO': 'health professional',
       'PSICÓLOGO': 'health professional',
       'ASSISTENTE SOCIAL': 'health professional',
       'ENFERMEIRO': 'health professional',
       'FISIOTERAPEUTA E TERAPEUTA OCUPACIONAL': 'health professional', 
       'FARMACEUTICO': 'health professional',
       'FARMACÊUTICO': 'health professional',
       'SOCIOLOGO': 'social scientist/historian',
       'SOCIÓLOGO': 'social scientist/historian',
       'CIENTISTA POLÍTICO': 'social scientist/historian',
       'HISTORIADOR': 'social scientist/historian',
       'CORRETOR DE IMÓVEIS, SEGUROS, TÍTULOS E VALORES': 'broker/economist/banker',
       'ECONOMISTA': 'broker/economist/banker',
       'BANCARIO E ECONOMIARIO': 'broker/economist/banker',
       'BANCÁRIO E ECONOMIÁRIO': 'broker/economist/banker',
       'LEILOEIRO, AVALIADOR E ASSEMELHADOS': 'broker/economist/banker',
       'SECURITARIO': 'broker/economist/banker',
       'ADMINISTRADOR': 'businessperson/entrepreneur',
       'EMPRESARIO': 'businessperson/entrepreneur',
       'EMPRESÁRIO': 'businessperson/entrepreneur',
       'GERENTE': 'businessperson/entrepreneur',
       'DIRETOR DE EMPRESAS': 'businessperson/entrepreneur',
       'ATLETA PROFISSIONAL E TECNICO EM DESPORTOS': 'athlete',
       'ATLETA PROFISSIONAL E TÉCNICO EM DESPORTOS': 'athlete',
       'SACERDOTE OU MEMBRO DE ORDEM OU SEITA RELIGIOSA': 'religious',
       'DIPLOMATA': 'diplomat',
       'RELAÇÕES-PÚBLICAS': 'public relations',
       'MAGISTRADO': 'judge/lawyer/prosecutor',
       'MEMBRO DO MINISTERIO PUBLICO': 'judge/lawyer/prosecutor',
       'MEMBRO DO MINISTÉRIO PÚBLICO': 'judge/lawyer/prosecutor',
       'GEÓLOGO': 'geologist',
       'GEÓGRAFO': 'geologist',
       'DONA DE CASA': 'houseperson/retired',
       'APOSENTADO (EXCETO SERVIDOR PUBLICO)': 'houseperson/retired',
       'APOSENTADO (EXCETO SERVIDOR PÚBLICO)': 'houseperson/retired',
       'ESCRITOR E CRÍTICO': 'writer',
       'ANALISTA DE SISTEMAS': 'computer scientist',
       'OTHER': 'other',
       'OUTROS': 'other',
       'NÃO INFORMADA': 'other',
    }

    if occupation in translation_table:
        return translation_table[occupation].lower()
    else:
        return 'other'

congresspeople['occupation'] = congresspeople['occupation'].fillna('other')
congresspeople['occupation'] = congresspeople['occupation'].apply(getoccupation)
congresspeople['occupation'].value_counts()

politician                     1604
other                          1023
businessperson/entrepreneur     382
judge/lawyer/prosecutor         334
dentist/medic                   208
engineer/architect              130
public servant                  107
university professor             80
broker/economist/banker          71
farmer                           70
teacher                          46
military                         45
salesman                         42
journalist                       41
health professional              30
veterinarian/agronomist          25
media worker                     25
blue collar                      24
religious                        23
student                          22
houseperson/retired              17
accountant                       16
social scientist/historian       11
actor/musician                   10
publicist                         4
veterinarian/agronomist/          4
biologist                         3
athlete                     

In [5]:
def getEductionLevel(education):
    education = education.upper()
    translation_table = { 
        'DOUTORADO': 'graduate',
        'MESTRADO': 'graduate',
        'DOUTORADO INCOMPLETO': 'graduate',
        'MESTRADO INCOMPLETO': 'undergraduate',
        'PÓS-GRADUAÇÃO': 'graduate',
        'SUPERIOR COMPLETO': 'undergraduate',
        'SUPERIOR': 'undergraduate',
        'SUPERIOR INCOMPLETO': 'high school',
        'ENSINO MÉDIO': 'high school',
        'ENSINO MÉDIO COMPLETO': 'high school',
        'MÉDIO COMPLETO': 'high school',
        'SECUNDÁRIO': 'high school',
        'ENSINO FUNDAMENTAL': 'elementary school',
        'ENSINO FUNDAMENTAL COMPLETO': 'elementary school',
        'FUNDAMENTAL COMPLETO': 'elementary school',
        'PRIMÁRIO': 'elementary school',
        'SECUNDÁRIO INCOMPLETO': 'elementary school',
        'ENSINO TÉCNICO': 'high school',
        'GINASIAL': 'high school',
        'ENSINO MÉDIO INCOMPLETO': 'elementary school',
        'MÉDIO INCOMPLETO': 'elementary school',
        'PRIMÁRIO INCOMPLETO': 'no education',
        'ENSINO FUNDAMENTAL INCOMPLETO': 'no education',
        'FUNDAMENTAL INCOMPLETO': 'no education',
        'LÊ E ESCREVE': 'no education',
        'NÃO INFORMADO': 'other',
        'OTHER': 'other'
        }
    if education in translation_table:
        return translation_table[education]
    else:
        logging.warning('Education not found: %s', education)
        return education
    
# Replace na escolaridade with the education field
congresspeople['escolaridade'] = congresspeople['escolaridade'].fillna(congresspeople['education'])
congresspeople['escolaridade'] = congresspeople['escolaridade'].fillna('other')
congresspeople['education'] = congresspeople['escolaridade'].apply(getEductionLevel)
congresspeople['education'].value_counts()

undergraduate        2954
graduate              693
high school           666
elementary school      90
no education            7
other                   1
Name: education, dtype: int64

In [6]:
congresspeople['gender'] = congresspeople['gender'].fillna(congresspeople['sexo'])
congresspeople['gender'] = congresspeople['gender'].apply(lambda x: 
                                                          'm' if x in ('MASCULINO', 'M')
                                                          else 'f' if x in ('FEMININO', 'F')
                                                          else 'other')

In [7]:
def getMaritalStatus(marital_status):
    marital_status = marital_status.upper()
    translation_table = {
        'CASADO(A)': 'married',
        'SOLTEIRO(A)': 'single',
        'DIVORCIADO(A)': 'divorced',
        'SEPARADO(A) JUDICIALMENTE': 'divorced',
        'VIÚVO(A)': 'widowed',
        'OUTROS': 'other',
        'NÃO INFORMADO': 'other',
        'OTHER': 'other'
    }
    if marital_status in translation_table:
        return translation_table[marital_status]
    else:
        logging.warning('Marital status not found: %s', marital_status)
        return marital_status
    
congresspeople['marital_status'] = congresspeople['marital_status'].fillna('other')
congresspeople['marital_status'] = congresspeople['marital_status'].apply(getMaritalStatus)
congresspeople['marital_status'].value_counts()

married     2619
other        845
single       490
divorced     414
widowed       43
Name: marital_status, dtype: int64

In [8]:
# Create authors dict. It must be an empty list for each proposal then we append the authors. One per year
authors_dict = {}

for year in range(2000, 2024):
    authors_dict[year] = {}
    for index, row in authors[authors['year'] == year].iterrows():
        if row['idProposicao'] in authors_dict[year]:
            authors_dict[year][row['idProposicao']].append(row['id'])
        else:
            authors_dict[year][row['idProposicao']] = [row['id']]

    # remake the dict and get only the proposals with more than one author and less than 10
    authors_dict[year] = {k: v for k, v in authors_dict[year].items() if len(v) > 1 and len(v) < 10}

In [9]:
congresspeople

,id,nomeCivil,nomeEleitoral,ufNascimento,escolaridade,dataNascimento,sexo,cpf,redeSocial,siglaUf,siglaPartido,idLegislatura,election_year,occupation,gender,education,marital_status,ethnicity,region
0,65551,RICARDO WAGNER DE CARVALHO LAGO,WAGNER LAGO,MA,Superior,1944-02-15,M,1706047304,[],MA,PDT,52,2002,judge/lawyer/prosecutor,m,undergraduate,married,NaN,Nordeste
1,139285,LÍDICE DA MATA E SOUZA,Lídice da Mata,BA,Superior,1956-03-12,F,14672049515,"['https://twitter.com/lidicedamata', 'https://...",BA,PSB,57,2022,politician,f,undergraduate,divorced,PARDA,Nordeste
2,139285,LÍDICE DA MATA E SOUZA,Lídice da Mata,BA,Superior,1956-03-12,F,14672049515,"['https://twitter.com/lidicedamata', 'https://...",BA,PSB,53,2006,politician,f,undergraduate,divorced,PARDA,Nordeste
3,139285,LÍDICE DA MATA E SOUZA,Lídice da Mata,BA,Superior,1956-03-12,F,14672049515,"['https://twitter.com/lidicedamata', 'https://...",BA,PSB,56,2018,politician,f,undergraduate,divorced,PARDA,Nordeste
4,196357,DEJORGE PATRICIO DA SILVA,DEJORGE PATRÍCIO,RJ,Ensino Médio,1974-06-18,M,3726063773,[],RJ,PRB,55,2014,politician,m,high school,single,PARDA,Sudeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,163831,FERNANDO ANTÔNIO CECILIANO JORDÃO,FERNANDO JORDÃO,RJ,Superior,1952-06-23,M,49752839720,[],RJ,PMDB,54,2010,businessperson/entrepreneur,m,undergraduate,married,NaN,Sudeste
4407,73720,César Antonio de Souza,CESAR SOUZA,SC,Secundário,1957-11-08,M,28951107949,[],SC,PSD,55,2014,businessperson/entrepreneur,m,high school,divorced,BRANCA,Sul
4408,180214,RAIMUNDO DINIZ ARAÚJO,Pastor Diniz,RR,Superior,1980-03-21,M,65822960278,[],RR,UNIÃO,57,2022,health professional,m,undergraduate,married,PARDA,Norte
4409,172029,FRANCISCO DAS CHAGAS FRANCILINO,FRANCISCO CHAGAS,SE,Superior,1956-04-21,M,76555283815,[],SP,PT,54,2010,politician,m,undergraduate,divorced,BRANCA,Sudeste


In [10]:
# Create age caracteristics. Separate by 10 years
def getAge(birthdate):
    birthdate = datetime.datetime.strptime(birthdate, '%Y-%m-%d')
    age = datetime.datetime.now() - birthdate
    age = int(age.days / 365)
    if age < 30:
        return 0
    if age < 40:
        return 1
    if age < 50:
        return 2
    if age < 60:
        return 3
    else:
        return 4
congresspeople['age'] = congresspeople['dataNascimento'].apply(getAge)

In [11]:
congresspeople = congresspeople[['id', 'idLegislatura', 'election_year', 'nomeEleitoral', 'education', 'gender', 'siglaUf', 'siglaPartido', 'region', 'occupation', 'marital_status', 'ethnicity', 'age']]
features = ['education', 'gender','siglaUf', 'siglaPartido', 'region', 'occupation', 'marital_status', 'ethnicity', 'age']
congresspeople

,id,idLegislatura,election_year,nomeEleitoral,education,gender,siglaUf,siglaPartido,region,occupation,marital_status,ethnicity,age
0,65551,52,2002,WAGNER LAGO,undergraduate,m,MA,PDT,Nordeste,judge/lawyer/prosecutor,married,NaN,4
1,139285,57,2022,Lídice da Mata,undergraduate,f,BA,PSB,Nordeste,politician,divorced,PARDA,4
2,139285,53,2006,Lídice da Mata,undergraduate,f,BA,PSB,Nordeste,politician,divorced,PARDA,4
3,139285,56,2018,Lídice da Mata,undergraduate,f,BA,PSB,Nordeste,politician,divorced,PARDA,4
4,196357,55,2014,DEJORGE PATRÍCIO,high school,m,RJ,PRB,Sudeste,politician,single,PARDA,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,163831,54,2010,FERNANDO JORDÃO,undergraduate,m,RJ,PMDB,Sudeste,businessperson/entrepreneur,married,NaN,4
4407,73720,55,2014,CESAR SOUZA,high school,m,SC,PSD,Sul,businessperson/entrepreneur,divorced,BRANCA,4
4408,180214,57,2022,Pastor Diniz,undergraduate,m,RR,UNIÃO,Norte,health professional,married,PARDA,2
4409,172029,54,2010,FRANCISCO CHAGAS,undergraduate,m,SP,PT,Sudeste,politician,divorced,BRANCA,4


In [23]:
congresspeople.age.unique()

array([4, 2, 3, 1, 0])

In [20]:
class NetworkBuilder():
    '''
    This class builds the network from the data.
    It starts with the congresspeople as nodes. Then we add edges between congresspeople
    who have co-authored a proposal. The weight of the edge is the number of proposals
    We also have other attributes for the edges, such as party affiliation, race, etc.
    '''

    def __init__(self, congresspeople, coauthors, features, period):
        self.G = nx.Graph()
        # self.add_nodes(congresspeople)
        # for feature in features:
        #     print(feature)
        #     relations = self.createRelation(congresspeople, feature)
        #     self.G.add_edges_from(relations, key=feature)
        self.add_nodes_attributes(congresspeople, features)
        self.add_edges_proposals(coauthors)
        self.save_graph(self.G, f'data/networks/{period}network.gpickle')

    def save_graph(self, G, path):
        '''
        Save the graph in a pickle file
        '''
        with open(path, 'wb') as f:
            pickle.dump(G, f)

    @staticmethod
    def createRelation(df, target_column):
        '''
        Given a dataframe and a target column, it returns a list of all tuples possible
        '''
        relations = []
        for relation in df[target_column].unique():
            target_df = df[df[target_column] == relation]
            for index, row in target_df.iterrows():
                for index, row in target_df.loc[index+1:].iterrows():
                    relations.append((row['id'], target_df.iloc[0]['id']))
        return relations

    def add_nodes(self, congresspeople):
        for row, data in congresspeople.iterrows():
            self.G.add_node(data['id'])

    def add_nodes_attributes(self, congresspeople, features):
        for row, data in congresspeople.iterrows():
            self.G.add_node(data['id'], **data[features].to_dict())

    def add_edges_proposals(self, authors_dict):
        for proposal in authors_dict:
            authors = authors_dict[proposal]
            coauthors_len = len(authors_dict[proposal])
            for i in range(coauthors_len):
                for j in range(i+1, coauthors_len):
                    if not self.G.has_edge(authors[i], authors[j]):
                        self.G.add_edge(authors[i], authors[j], weight=1)
                    else:
                        self.G[authors[i]][authors[j]]['weight'] += 1

    def getGraph(self):
        return self.G

In [ ]:
networks_yearly = {}
for year in range(2000, 2024):
    print(year)
    idLegislatura = (year - 1999) // 4 + 51
    congress = congresspeople[congresspeople['idLegislatura'] == idLegislatura]
    network = NetworkBuilder(congress, authors_dict[year], features, year)
    networks_yearly[year] = network.getGraph()

In [ ]:
networks_termly = {}
for idLegislatura in range(51, 58):
    print(idLegislatura)
    election_year = {57: 2022, 56: 2018, 55: 2014,
                 54: 2010, 53: 2006, 52: 2002,
                 51: 1998}
    congress = congresspeople[congresspeople['idLegislatura'] == idLegislatura]
    
    years = list(range(election_year[55] + 1, election_year[55] + 5))
    authors_dict_term = {}
    for year in years:
        authors_dict_term.update(authors_dict[year])

    network = NetworkBuilder(congress, authors_dict_term, features, idLegislatura)
    networks_termly[idLegislatura] = network.getGraph()